# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 7. Support Vector Machine (SVM) Lanjutan


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [1]:
#sudah di download

Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [2]:
import pandas as pd  
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [3]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [4]:
from sklearn.model_selection import train_test_split  
data_latih,data_uji = train_test_split(data,test_size=0.2)

In [5]:
data_uji.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
121,5.6,2.8,4.9,2.0,Virginica
3,4.6,3.1,1.5,0.2,Setosa
102,7.1,3.0,5.9,2.1,Virginica
138,6.0,3.0,4.8,1.8,Virginica
131,7.9,3.8,6.4,2.0,Virginica


Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [6]:
print(data_latih.shape[0])
print(data_uji.shape[0]) 

120
30


Pisahkan label/kelas dari data uji menjadi sebuah variabel bernama **label_uji**

In [7]:
label_uji = data_uji.pop('variety')

## 3) Pembentukan data latih one-vs-rest

Metode one-vs-rest memerlukan tiga jenis data latih yang diperlukan untuk melatih tiga SVM yang berbeda pada dataset Iris. Fungsi **buat_trainingset** digunakan untuk membentuk tiga dataset tersebut.

In [8]:
def buat_trainingset(dataset):
  trainingset = {}
  kolom_kelas = dataset.columns[-1]
  list_kelas = dataset[kolom_kelas].unique()
  for kelas in list_kelas:
    data_temp = dataset.copy(deep=True)
    data_temp[kolom_kelas] = data_temp[kolom_kelas].map({kelas:1})
    data_temp[kolom_kelas] = data_temp[kolom_kelas].fillna(-1)
    trainingset[kelas] = data_temp
  return trainingset

Gunakan fungsi **buat_trainingset** untuk membentuk data latih dengan nama variabel **trainingset** yang akan digunakan pada proses training. 

In [9]:
trainingset = buat_trainingset(data_latih)

Tampilkan isi **trainingset** agar Anda dapat memahami struktur dari variabel tersebut.

In [10]:
print(trainingset)

{'Versicolor':      sepal.length  sepal.width  petal.length  petal.width  variety
85            6.0          3.4           4.5          1.6      1.0
92            5.8          2.6           4.0          1.2      1.0
9             4.9          3.1           1.5          0.1     -1.0
106           4.9          2.5           4.5          1.7     -1.0
95            5.7          3.0           4.2          1.2      1.0
..            ...          ...           ...          ...      ...
89            5.5          2.5           4.0          1.3      1.0
46            5.1          3.8           1.6          0.2     -1.0
51            6.4          3.2           4.5          1.5      1.0
128           6.4          2.8           5.6          2.1     -1.0
50            7.0          3.2           4.7          1.4      1.0

[120 rows x 5 columns], 'Setosa':      sepal.length  sepal.width  petal.length  petal.width  variety
85            6.0          3.4           4.5          1.6     -1.0
92          

## 4) Pembentukan SVM Biner

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [11]:
def hitung_cost_gradient(W, X, Y, regularization):
  jarak = 1 - (Y * np.dot(X, W))
  dw = np.zeros(len(W))
  if max(0, jarak) == 0:
    di = W
  else: di = W - (regularization * Y * X)
  dw += di
  return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD 

In [12]:
from sklearn.utils import shuffle
def sgd(data_latih,label_latih,learning_rate = 0.000001,max_epoch=1000,regularization=10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1,max_epoch):
    X,Y = shuffle(data_latih,label_latih,random_state=101)
    for index,x in enumerate(X):
      delta=hitung_cost_gradient(bobot,x,Y[index],regularization)
      bobot=bobot-(learning_rate*delta)
    return bobot

## 5) Proses Training

Proses training dilakukan dengan memanggil fungsi **sgd** berulang kali sesuai banyaknya kelas yang ada pada data. Dengan demikian, proses training menghasilkan bobot sebanyak kelas yang ada pada dataset. Buatlah fungsi bernama **training** yang digunakan untuk melakukan proses training one-vs-rest

In [13]:
def training(trainingset):
  list_kelas = trainingset.keys()
  w = {}
  for kelas in list_kelas:
    data_latih = trainingset[kelas]
    label_latih = data_latih.pop(data_latih.columns[-1])
    w[kelas] = sgd(data_latih, label_latih)
  return w

Lakukan proses training dengan memanggil fungsi **training** dan menempatkan hasilnya pada variabel **W**

In [14]:
W = training(trainingset) 

Tampilkan isi variabel **W**

In [15]:
print(W)

{'Versicolor': array([-0.13298893, -0.24797973,  0.08299332,  0.00599854]), 'Setosa': array([ 0.176995  ,  0.45697098, -0.60595373, -0.27497989]), 'Virginica': array([-0.22998019, -0.32397564,  0.4119735 ,  0.23398512])}


## 6) Proses *testing* biner
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [16]:
def testing(W, data_uji):
  prediksi = np.array([])
  for i in range(data_uji.shape[0]):
    y_prediksi = np.sign(np.dot(W, data_uji.to_numpy()[i]))
    prediksi = np.append(prediksi, y_prediksi)
  return prediksi

## TUGAS
Pada tugas kali ini Anda mendefinisikan proses testing pada metode one-vs-rest. Proses testing pada metode one-vs-rest dilakukan dengan memanggil proses testing biner untuk setiap **value** pada dictionary **W**. Kelas pada sebuah data latih adalah **key** pada dictionary **W** yang memiliki nilai prediksi **1**. Lengkapi fungsi **testing_onevsrest** di bawah ini. Output dari fungsi tersebut adalah list nama kelas hasil prediksi.

In [17]:
def testing_onevsrest(W,data_uji):
    hasil_prediksi = []
    list_kelas = W.keys()
    hasil = {kelas:[] for kelas in list_kelas}
    for i in range (data_uji.shape[0]):
        for kelas, w in W.items():
            hasil[kelas].append(np.sign(np.dot(w,data_uji.to_numpy()[i])))
    for i in range(data_uji.shape[0]):
        max_value = -9999
        max_key = ''
        for kelas, nilai in hasil.items():
            if nilai[i] > max_value:
                max_value = nilai[i]
                max_key = kelas
        hasil_prediksi.append(max_key)
    return hasil_prediksi

In [18]:
prediksi = testing_onevsrest(W,data_uji)

In [19]:
prediksi

['Virginica',
 'Setosa',
 'Virginica',
 'Virginica',
 'Virginica',
 'Virginica',
 'Virginica',
 'Setosa',
 'Versicolor',
 'Versicolor',
 'Setosa',
 'Versicolor',
 'Versicolor',
 'Virginica',
 'Setosa',
 'Setosa',
 'Versicolor',
 'Virginica',
 'Versicolor',
 'Virginica',
 'Setosa',
 'Virginica',
 'Virginica',
 'Setosa',
 'Virginica',
 'Versicolor',
 'Versicolor',
 'Versicolor',
 'Virginica',
 'Versicolor']

Berapa banyak data latih yang berhasil diprediksi dengan benar?

In [20]:
print(f"Total data yang sama dengan label sebenarnya: {np.sum(prediksi == label_uji)} dari {data_uji.shape[0]} data uji")


Total data yang sama dengan label sebenarnya: 29 dari 30 data uji
